In [202]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import json
import urllib.request as ulr
from googleplaces import GooglePlaces, types, lang
YOUR_API_KEY = #<Put your Google API key here as a string>

In [7]:
pd.set_option('max_rows', 10)

## Load health inspection and weather data

In [9]:
violations_df = pd.read_csv('../Geocoding/health_inspect_cleaned_loc_added.csv', index_col=0) # file is in GitHub
dba_df = pd.read_csv('camis_dba.csv', index_col = 0)
weather_df = pd.read_csv('../Weather/weather_data.csv') # file is in GitHub

### Add temperature and humidity columns to health inspection data

In [10]:
violations_df['3-day temp'] = list(weather_df['3-day temp'])
violations_df['3-day humidity'] = list(weather_df['3-day humidity'])
violations_df = violations_df.merge(dba_df, left_index=True, right_index=True)
violations_df['search_info'] = violations_df.names.str.cat(violations_df.address2, sep=' ')
violations_df

,crit_violations,non_crit_violations,earliest_inspection,latest_inspection,second_latest_inspection,zipcode,boro,cuisine,address,num_inspections,...,time_since_last_inspection,time_since_first_inspection,crit_v_2plus,address2,latitude,longitude,3-day temp,3-day humidity,names,search_info
30075445,8.0,4.0,2013-08-14,2016-02-18,2015-02-09,10462,BRONX,Bakery,1007 MORRIS PARK AVE,5,...,374,918,0,"1007 MORRIS PARK AVE, NY",40.848537,-73.856123,33.666667,50.000000,morris park bake shop,"morris park bake shop 1007 MORRIS PARK AVE, NY"
30112340,14.0,10.0,2014-06-05,2016-10-27,2016-10-03,11225,BROOKLYN,Hamburgers,469 FLATBUSH AVENUE,9,...,24,875,0,"469 FLATBUSH AVENUE, NY",40.662930,-73.961726,74.000000,72.000000,wendy's,"wendy's 469 FLATBUSH AVENUE, NY"
30191841,4.0,7.0,2013-07-22,2016-05-31,2015-09-21,10019,MANHATTAN,Irish,351 WEST 57 STREET,5,...,253,1044,0,"351 WEST 57 STREET, NY",40.767821,-73.984981,35.000000,69.333333,dj reynolds pub and restaurant,dj reynolds pub and restaurant 351 WEST 57 S...
40356018,1.0,7.0,2013-06-05,2016-05-16,2015-06-05,11224,BROOKLYN,American,2780 STILLWELL AVENUE,4,...,346,1076,0,"2780 STILLWELL AVENUE, NY",40.579526,-73.982426,55.000000,63.333333,riviera caterer,"riviera caterer 2780 STILLWELL AVENUE, NY"
40356151,13.0,5.0,2014-04-11,2016-05-14,2015-05-29,11369,QUEENS,American,8825 ASTORIA BOULEVARD,7,...,351,764,0,"8825 ASTORIA BOULEVARD, NY",40.772354,-73.931502,43.000000,61.666667,brunos on the boulevard,brunos on the boulevard 8825 ASTORIA BOULEVARD...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50060427,10.0,4.0,2017-03-07,2017-04-06,2017-03-20,11217,BROOKLYN,Bagels/Pretzels,73 LAFAYETTE AVE,3,...,17,30,1,"73 LAFAYETTE AVE, NY",40.687281,-73.975386,57.000000,69.000000,la bagel delight,"la bagel delight 73 LAFAYETTE AVE, NY"
50060496,7.0,4.0,2017-02-27,2017-03-31,2017-02-27,11232,BROOKLYN,Delicatessen,4102 3RD AVE,2,...,32,32,1,"4102 3RD AVE, NY",40.652797,-74.009697,50.000000,42.666667,cousins deli restaurant,"cousins deli restaurant 4102 3RD AVE, NY"
50060695,7.0,4.0,2017-04-04,2017-04-10,2017-04-04,11432,QUEENS,Pizza,16417 JAMAICA AVE,2,...,6,6,0,"16417 JAMAICA AVE, NY",40.723242,-73.728041,53.666667,62.000000,2 bros pizza,"2 bros pizza 16417 JAMAICA AVE, NY"
50060807,2.0,2.0,2017-03-09,2017-03-20,2017-03-09,10467,BRONX,Chinese,2861 WHITE PLAINS RD,2,...,11,11,0,"2861 WHITE PLAINS RD, NY",40.867355,-73.867400,78.000000,68.000000,century chinese food,"century chinese food 2861 WHITE PLAINS RD, NY"


In [2]:


#Grabbing and parsing the JSON data
def GoogPlac(lat,lng,radius,key):
    #making the url
    AUTH_KEY = key
    LOCATION = str(lat) + "," + str(lng)
    RADIUS = radius
    MyUrl = ('https://maps.googleapis.com/maps/api/place/nearbysearch/json'
             '?location=%s'
             '&radius=%s'
             #'&types=%s'
             '&sensor=false&key=%s') % (LOCATION, RADIUS,AUTH_KEY)
    #grabbing the JSON result
    response = ulr.urlopen(MyUrl)
    data = response.read().decode("utf-8")
    data = json.loads(data)
    return data
def GoogPlacText(loc,radius,key):
    #making the url
    AUTH_KEY = key
    LOCATION = loc.replace(' ', '+')
    RADIUS = radius
    MyUrl = ('https://maps.googleapis.com/maps/api/place/textsearch/json'
             '?query=%s'
             '&radius=%s'
             '&sensor=false&key=%s') % (LOCATION, RADIUS,AUTH_KEY)
    #grabbing the JSON result
    response = ulr.urlopen(MyUrl)
    data = response.read().decode("utf-8")
    data = json.loads(data)
    return data

def GoogPlacID(ID,key):
    AUTH_KEY = key
    PLACE_ID = ID
    MyUrl = ('https://maps.googleapis.com/maps/api/place/details/json'
             '?placeid=%s'
             '&key=%s') % (PLACE_ID,AUTH_KEY)
    #grabbing the JSON result
    response = ulr.urlopen(MyUrl)
    data = response.read().decode("utf-8")
    data = json.loads(data)
    return data


In [29]:
pl_names_official = []
pl_names_google = []
pl_ratings = []
search_failures = []
failure_index = []
ct = 0

In [60]:
from tqdm import tqdm 
import time

#Adjust the range as needed for missing values
for i in tqdm(np.arange(14000,violations_df.shape[0])):
    try:
        rest_name = violations_df['names'].iloc[i]
        loc = violations_df['search_info'].iloc[i]
        query = GoogPlacText(loc, 50, YOUR_API_KEY)
        google_name = query['results'][0]['name'].lower()
        google_rating = query['results'][0]['rating']
        pl_names_official.append(rest_name)
        pl_names_google.append(google_name)
        pl_ratings.append(google_rating)


    except:
        ct = ct + 1
        pl_names_official.append(violations_df['names'].iloc[i])
        pl_names_google.append('SEARCH FAILURE')
        pl_ratings.append(np.nan)
        search_failures.append(violations_df['search_info'].iloc[i])
        failure_index.append(i)
        if np.mod(ct,25) == 0:
            print('Failure ' + str(ct) + ': ' + str(violations_df['search_info'].iloc[i]))

   

  2%|▏         | 140/8330 [02:17<1:54:38,  1.19it/s]

Failure 1050: blanca/severed heads 261 MOORE, NY


  5%|▌         | 419/8330 [06:49<2:04:13,  1.06it/s]

Failure 1075: the p insula new york/ clement/ the gotham lounge/ salon de ning 700 FIFTH AVE, NY


  8%|▊         | 652/8330 [10:32<1:37:13,  1.32it/s]

...

 97%|█████████▋| 8112/8330 [2:08:50<03:04,  1.18it/s]

Failure 1825: clear lake 4429 9TH ST, NY


 99%|█████████▉| 8253/8330 [2:11:04<00:59,  1.30it/s]

Failure 1850: tbaar 13620 ROOSEVELT AVE, NY


100%|██████████| 8330/8330 [2:12:17<00:00,  1.09it/s]


In [63]:
print(len(pl_names_official))
print(len(pl_names_google))
print(len(pl_ratings))
print(len(search_failures))
print(len(failure_index))

22645
22330
22330
1866
1866


In [62]:
df_ratings = pd.DataFrame([violations_df['search_info'].values, pl_names_google, pl_ratings]).T
df_ratings = df_ratings.rename(index=str, columns={0: 'search_info', 1: 'google_name', 2: 'rating'})
df_ratings.index = violations_df.index
df_ratings.to_csv('ratings0_23329.csv')

### Rerun search on the previous failures after removing special characters

In [131]:
names_official_fail = []
names_google_fail = []
ratings_fail = []
double_failure = []
double_failure_index = []
ct_fail = 0

In [132]:
for i in tqdm(np.arange(0,len(failure_index))):
    try:
        rest_name = violations_df['names'].iloc[failure_index[i]]
        loc = violations_df['search_info'].iloc[failure_index[i]]
        loc = loc.replace('# ', '')
        loc = loc.replace('.', '')
        loc = loc.replace("'", '')
        loc = loc.replace('&', '')
        loc = loc.replace('  ', ' ')
        loc = loc.replace('/', ' ')
        query = GoogPlacText(loc, 3000, YOUR_API_KEY)
        google_name = query['results'][0]['name'].lower()
        google_rating = query['results'][0]['rating']
        names_official_fail.append(rest_name)
        names_google_fail.append(google_name)
        ratings_fail.append(google_rating)


    except:
        ct_fail = ct_fail + 1
        names_official_fail.append(violations_df['names'].iloc[failure_index[i]])
        names_google_fail.append('SEARCH FAILURE')
        ratings_fail.append(np.nan)
        double_failure.append(violations_df['search_info'].iloc[failure_index[i]])
        double_failure_index.append(failure_index[i])
        if np.mod(ct_fail,25) == 0:
            print('Failure ' + str(ct_fail) + ': ' + str(violations_df['search_info'].iloc[failure_index[i]]))


  3%|▎         | 48/1866 [00:35<16:24,  1.85it/s]

Failure 25: rosa #2 pizzeria 7559 METROPOLITAN AVENUE, NY


  4%|▍         | 77/1866 [00:58<25:31,  1.17it/s]

Failure 50: ing forman selz 230 PARK AVENUE, NY


  6%|▌         | 111/1866 [01:27<31:43,  1.08s/it]

...

 98%|█████████▊| 1829/1866 [25:06<00:24,  1.48it/s]

Failure 1475: zero lounge 1472 OGDEN AVE, NY


100%|█████████▉| 1858/1866 [25:27<00:07,  1.10it/s]

Failure 1500: + 81 gallery new york 167 ELIZABETH ST, NY


100%|██████████| 1866/1866 [25:35<00:00,  1.07it/s]


In [139]:
for i in range(len(failure_index)):
    pl_names_google[failure_index[i]] = names_google_fail[i]
    pl_ratings[failure_index[i]] = ratings_fail[i]

In [147]:
df_ratings = pd.DataFrame([violations_df['search_info'].values, pl_names_google, pl_ratings]).T
df_ratings = df_ratings.rename(index=str, columns={0: 'search_info', 1: 'google_name', 2: 'rating'})
df_ratings.index = violations_df.index
df_ratings.to_csv('ratings0_23329.csv')

### For restaurants that are missing ratings, we first try to impute based on the mean at the zipcode level
#### If this fails, we use the overall mean, creating indicator variables at both stages

In [199]:
df_merged = violations_df.merge(df_ratings.drop(['search_info', 'google_name'], axis=1), left_index=True, right_index=True)
df_merged['rating'] = pd.to_numeric(df_merged['rating'])
mean_rating = df_merged['rating'].mean()
by_zip = df_merged.groupby('zipcode').mean()
df_merged['rating_null'] = df_merged['rating'].isnull()*1
nan_index = df_merged.index[df_merged['rating'].isnull()]
for i in tqdm(range(len(nan_index))):
    df_merged.loc[nan_index[i],'rating'] = by_zip.ix[df_merged.ix[nan_index[i]]['zipcode']].rating
df_merged['rating_null_zip'] = df_merged['rating'].isnull()*1
nan_index_zip = df_merged.index[df_merged['rating'].isnull()]
for i in tqdm(range(len(nan_index_zip))):
    df_merged.loc[nan_index_zip[i],'rating'] = mean_rating
df_merged

100%|██████████| 3/3 [00:00<00:00, 317.16it/s]


,crit_violations,non_crit_violations,earliest_inspection,latest_inspection,second_latest_inspection,zipcode,boro,cuisine,address,num_inspections,...,address2,latitude,longitude,3-day temp,3-day humidity,names,search_info,rating,rating_null,rating_null_zip
30075445,8.0,4.0,2013-08-14,2016-02-18,2015-02-09,10462,BRONX,Bakery,1007 MORRIS PARK AVE,5,...,"1007 MORRIS PARK AVE, NY",40.848537,-73.856123,33.666667,50.000000,morris park bake shop,"morris park bake shop 1007 MORRIS PARK AVE, NY",4.800000,0,0
30112340,14.0,10.0,2014-06-05,2016-10-27,2016-10-03,11225,BROOKLYN,Hamburgers,469 FLATBUSH AVENUE,9,...,"469 FLATBUSH AVENUE, NY",40.662930,-73.961726,74.000000,72.000000,wendy's,"wendy's 469 FLATBUSH AVENUE, NY",3.400000,0,0
30191841,4.0,7.0,2013-07-22,2016-05-31,2015-09-21,10019,MANHATTAN,Irish,351 WEST 57 STREET,5,...,"351 WEST 57 STREET, NY",40.767821,-73.984981,35.000000,69.333333,dj reynolds pub and restaurant,dj reynolds pub and restaurant 351 WEST 57 S...,3.500000,0,0
40356018,1.0,7.0,2013-06-05,2016-05-16,2015-06-05,11224,BROOKLYN,American,2780 STILLWELL AVENUE,4,...,"2780 STILLWELL AVENUE, NY",40.579526,-73.982426,55.000000,63.333333,riviera caterer,"riviera caterer 2780 STILLWELL AVENUE, NY",4.800000,0,0
40356151,13.0,5.0,2014-04-11,2016-05-14,2015-05-29,11369,QUEENS,American,8825 ASTORIA BOULEVARD,7,...,"8825 ASTORIA BOULEVARD, NY",40.772354,-73.931502,43.000000,61.666667,brunos on the boulevard,brunos on the boulevard 8825 ASTORIA BOULEVARD...,3.800000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50060427,10.0,4.0,2017-03-07,2017-04-06,2017-03-20,11217,BROOKLYN,Bagels/Pretzels,73 LAFAYETTE AVE,3,...,"73 LAFAYETTE AVE, NY",40.687281,-73.975386,57.000000,69.000000,la bagel delight,"la bagel delight 73 LAFAYETTE AVE, NY",4.100000,0,0
50060496,7.0,4.0,2017-02-27,2017-03-31,2017-02-27,11232,BROOKLYN,Delicatessen,4102 3RD AVE,2,...,"4102 3RD AVE, NY",40.652797,-74.009697,50.000000,42.666667,cousins deli restaurant,"cousins deli restaurant 4102 3RD AVE, NY",4.800000,0,0
50060695,7.0,4.0,2017-04-04,2017-04-10,2017-04-04,11432,QUEENS,Pizza,16417 JAMAICA AVE,2,...,"16417 JAMAICA AVE, NY",40.723242,-73.728041,53.666667,62.000000,2 bros pizza,"2 bros pizza 16417 JAMAICA AVE, NY",3.792806,1,0
50060807,2.0,2.0,2017-03-09,2017-03-20,2017-03-09,10467,BRONX,Chinese,2861 WHITE PLAINS RD,2,...,"2861 WHITE PLAINS RD, NY",40.867355,-73.867400,78.000000,68.000000,century chinese food,"century chinese food 2861 WHITE PLAINS RD, NY",3.700000,0,0


### Save resulting CSV with ratings

In [200]:
df_merged.to_csv('Health_Insp_weather_loc_rating.csv')

,crit_violations,non_crit_violations,zipcode,num_inspections,crit_violations_recent_inspect,non_crit_violations_recent_inspect,crit_violations_train,non_crit_violations_train,num_inspections_train,average_crit_v_train,...,time_since_last_inspection,time_since_first_inspection,crit_v_2plus,latitude,longitude,3-day temp,3-day humidity,rating,rating_null,rating_null_zip
count,22330.000000,22330.000000,22330.000000,22330.000000,22330.000000,22330.000000,22330.000000,22330.000000,22330.000000,22330.000000,...,22330.000000,22330.000000,22330.000000,22307.000000,22307.000000,20813.000000,20524.000000,22330.000000,22330.000000,22330.000000
mean,10.123780,7.033901,10666.568473,6.058979,1.486028,1.438513,8.637752,5.595387,5.058979,1.549885,...,219.461397,798.188133,0.345320,40.818079,-73.850940,53.713953,60.996914,4.016701,0.067532,0.000134
std,7.344702,4.382478,597.300235,2.849434,0.816336,0.645370,7.156231,4.276052,2.849434,0.756661,...,153.311738,294.366365,0.475483,1.040092,3.534032,14.916156,11.699470,0.558839,0.250948,0.011590
min,1.000000,1.000000,10001.000000,2.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,...,1.000000,2.000000,0.000000,-33.991932,-79.313640,20.333333,27.666667,1.000000,0.000000,0.000000
25%,4.000000,4.000000,10022.000000,4.000000,1.000000,1.000000,3.000000,2.000000,3.000000,1.000000,...,47.000000,719.000000,0.000000,40.674369,-73.987695,42.333333,53.333333,3.800000,0.000000,0.000000
50%,8.000000,6.000000,10466.000000,6.000000,1.000000,1.000000,7.000000,5.000000,5.000000,1.500000,...,210.000000,908.000000,0.000000,40.730360,-73.960202,51.333333,60.666667,4.100000,0.000000,0.000000
75%,14.000000,9.000000,11229.000000,8.000000,2.000000,2.000000,12.000000,8.000000,7.000000,2.000000,...,373.000000,1020.000000,1.000000,40.769978,-73.896635,63.666667,69.000000,4.400000,0.000000,0.000000
max,65.000000,32.000000,11697.000000,20.000000,10.000000,6.000000,59.000000,31.000000,19.000000,7.000000,...,1071.000000,1095.000000,1.000000,67.261060,164.861367,89.000000,92.000000,5.000000,1.000000,1.000000
